This notebook will use the robin_stocks.robinhood.export functions which write all orders to a csv file.  We will do this for stock, options and crypto orders

This first cell will pull the stock orders using the export_completed_stock_orders function
https://robin-stocks.readthedocs.io/en/latest/robinhood.html#robin_stocks.robinhood.export.export_completed_stock_orders

In [42]:
%pip install robin_stocks pandas numpy ipykernel

Note: you may need to restart the kernel to use updated packages.


This cell logs on to the robinhood web service endpoint

In [12]:
import robin_stocks.robinhood as r
import pandas as pd

# Prompt for email address
email = input("Please enter your Robinhood email address: ")

login = r.login(email)


Now that we are logged in lets get all the stock orders  

In [50]:
# # Get all stock orders
r.export_completed_stock_orders('/', file_name='../output/stock_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...


Now that we have the stock orders lets parse them, caclute cost of each order, and sum up the total for cumulative stock profit and loss.  Stock buys will be updated with a negative cost and sells will be a positive cost.   For example I buy 10 shares of AMD for $10 a share, this will cost me $100 and will reduce my balance by that much.  Then if I decide to sell my 10 AMD shares and get filled at $11 share for a total of $110, which will be added to my balance.  The total P&L for that trade would be +$10 since I sold the position for $10 more than I paid for it.  Conversly if I sold the 10 AMD and got filled at $9 a share for a total of $90, which will be added back to my cash balance.  The total P&L for this trade would be -$10 since I sold the position for $10 less than I paid for it.

In [102]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/stock_output.csv')

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'],  errors='coerce', utc=True)

#df = df[df['date'] >= '2019-02-12']

# Sort the DataFrame based on the date column
df = df.sort_values(by=['symbol','date'])

# Assuming 'quantity', 'average_price', and 'type' columns exist
df['cost'] = df['quantity'] * df['average_price']

# Adjust the cost based on the type of order
df.loc[df['side'] == 'buy', 'cost'] *= -1

print(df)
total_stock_cost = df['cost'].sum()
print(total_stock_cost)

df.to_csv('../output/processed_stock_orders.csv', index=False)


    symbol                             date order_type  side  fees  quantity  \
390   AAPL 2017-04-04 18:41:43.529000+00:00     market   buy  0.00       1.0   
372   AAPL 2017-04-10 15:43:06.897000+00:00     market  sell  0.02       1.0   
280   AAPL 2017-11-06 15:30:09.183000+00:00     market   buy  0.00       1.0   
268   AAPL 2017-11-13 16:03:16.685000+00:00     market   buy  0.00       1.0   
266   AAPL 2017-11-13 19:06:33.486000+00:00     market   buy  0.00       1.0   
..     ...                              ...        ...   ...   ...       ...   
364    XOP 2017-04-11 13:30:00.553000+00:00     market  sell  0.02       2.0   
282    XOP 2017-11-01 17:42:02.880000+00:00      limit   buy  0.00       1.0   
277    XOP 2017-11-06 17:31:33.778000+00:00     market  sell  0.02       1.0   
241    XSD 2017-11-27 14:57:07.327000+00:00     market   buy  0.00       2.0   
234    XSD 2017-11-28 15:47:24.601000+00:00     market  sell  0.02       2.0   

     average_price    cost  
390       

Now lets get all crypto orders

In [104]:
# # Get all crypto orders
r.export_completed_crypto_orders('/', file_name='../output/crypto_output.csv')

Now lets do the same thing we did with the stock orders to the crypto orders

In [105]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/crypto_output.csv')

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'],  errors='coerce', utc=True)

#df = df[df['date'] >= '2019-02-12']

# Sort the DataFrame based on the date column
df = df.sort_values(by=['symbol','date'])

# Assuming 'quantity', 'average_price', and 'type' columns exist
df['cost'] = df['quantity'] * df['average_price']

# Adjust the cost based on the type of order
df.loc[df['side'] == 'buy', 'cost'] *= -1

print(df)
total_crypto_cost = df['cost'].sum()
print(total_crypto_cost)

df.to_csv('../output/processed_crypto_orders.csv', index=False)


     symbol                             date order_type  side  fees  \
22   BTCUSD 2018-08-05 06:49:50.771000+00:00     market   buy   0.0   
21   BTCUSD 2018-08-12 07:49:48.270000+00:00     market  sell   0.0   
9    BTCUSD 2023-12-14 15:06:24.232000+00:00     market  sell   0.0   
8    BTCUSD 2024-06-24 19:04:34.507719+00:00     market   buy   0.0   
7    BTCUSD 2024-07-04 14:30:08.042036+00:00     market   buy   0.0   
6    BTCUSD 2024-07-05 02:53:50.275000+00:00      limit   buy   0.0   
3    BTCUSD 2024-08-12 23:20:08.899000+00:00     market  sell   0.0   
15  DOGEUSD 2018-09-09 02:23:25.038000+00:00     market   buy   0.0   
14  DOGEUSD 2018-09-10 16:45:36.039000+00:00     market  sell   0.0   
11  DOGEUSD 2018-10-09 11:18:51.460000+00:00     market   buy   0.0   
10  DOGEUSD 2018-10-09 11:19:30.680000+00:00     market  sell   0.0   
34   ETHUSD 2018-06-06 20:23:57.487000+00:00      limit   buy   0.0   
33   ETHUSD 2018-06-29 20:23:59.053000+00:00      limit   buy   0.0   
32   E

Now lets get all the options orders.

In [84]:
# # Get all option orders
r.export_completed_option_orders('/', file_name='../output/options_output.csv')

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...


This groups spread order by order_created_at and then calculates total cost based on debit and credit direction.  Spreads orders show up multiple time (number of legs) with the total price and quantity listed on each order, so we group them and only sum them up once per opening and closing spread order

In [108]:
import pandas as pd

# Load your spreadsheet
df = pd.read_csv('../output/options_output.csv')

# Convert 'order_created_at' to datetime
df['order_created_at'] = pd.to_datetime(df['order_created_at'])
df['order_created_at'] = df['order_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filter for the year 2024
#df = df[df['order_created_at'].str.startswith('2024')]

# Select only the columns we want
df = df[['order_created_at', 'chain_symbol', 'expiration_date', 
         'strike_price', 'option_type', 'direction', 'order_quantity', 
         'order_type', 'opening_strategy', 'closing_strategy', 'price', 'processed_quantity']]

# Group by 'order_created_at' and aggregate
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: list(sorted(x, key=abs, reverse=True)),  # Create a list of strike price
    'option_type': 'first',
    'direction': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'order_quantity': 'first'
}).reset_index()

# Sort the DataFrame
aggregated_df = aggregated_df.sort_values(by=['chain_symbol', 'expiration_date'])

# Calculate cost
aggregated_df['cost'] = aggregated_df['price'] * aggregated_df['order_quantity'] * 100

# Adjust the cost based on the direction
aggregated_df['cost'] = aggregated_df.apply(
    lambda x: -x['cost'] if x['direction'] == 'debit' else x['cost'],
    axis=1
)      

total_option_cost = aggregated_df['cost'].sum()
print(f"Total option cost: ${total_option_cost:.2f}")
print(aggregated_df['order_created_at'].count())
print(aggregated_df['order_quantity'].sum())

# Save the results
aggregated_df.to_csv('../output/options_orders_parsed.csv', index=False)

# Count rows where 'opening_strategy' is not None
total_opening_strategy_count = aggregated_df['opening_strategy'].notna().sum()
total_closing_strategy_count = aggregated_df['closing_strategy'].notna().sum()


# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# spread_closing_count = df['closing_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# spread_opening_count = df['opening_strategy'].str.contains('spread|iron', case=False, na=False).sum()

# # Count rows where 'closing_strategy' contains 'spread' or 'iron'
# non_spread_closing_count = total_closing_strategy_count - spread_closing_count

# # Count rows where 'opening_strategy' contains 'spread' or 'iron'
# non_spread_opening_count = total_opening_strategy_count - spread_opening_count
# Calculate total order quantity for opening_strategy
total_opening_quantity = aggregated_df['order_quantity'][aggregated_df['opening_strategy'].notna()].sum()
print(f"Total opening strategy quantity: {total_opening_quantity}")

# Calculate total order quantity for closing_strategy
total_closing_quantity = aggregated_df['order_quantity'][aggregated_df['closing_strategy'].notna()].sum()
print(f"Total closing strategy quantity: {total_closing_quantity}")


#Print the counts
print(f"Opening strategy rows: {total_opening_strategy_count}")
print(f"Closing strategy rows: {total_closing_strategy_count}")
# print(f"Opening strategy rows with 'spread' or 'iron': {spread_opening_count}")
# print(f"Closing strategy rows with 'spread' or 'iron': {spread_closing_count}")
# print(f"Opening strategy rows not with 'spread' or 'iron': {non_spread_opening_count}")
# print(f"Closing strategy rows not with 'spread' or 'iron': {non_spread_closing_count}")

Total option cost: $-3339.00
865
1607.0
Total opening strategy quantity: 1001.0
Total closing strategy quantity: 607.0
Opening strategy rows: 540
Closing strategy rows: 326


# AAPL Spread Order Analysis

This cell analyzes AAPL spread orders by:

1. Filtering for AAPL spread/iron condor orders
2. Pairing opening orders with corresponding closing orders
3. Handling partial closings and multiple closing orders per opening order
4. Calculating total opened and closed quantities
5. Identifying unpaired (still open) orders

The analysis provides:
- Count of paired and unpaired orders
- Detailed view of first few paired orders (open and close details)
- List of unpaired (open) orders
- Summary of total opened, closed, and remaining open quantities

This helps in understanding the current state of AAPL spread positions, including fully closed, partially closed, and still open trades.

In [92]:
# Assuming we're using the aggregated_df from the previous cell

# Filter for AAPL spread orders
aapl_spreads = aggregated_df[
    (aggregated_df['chain_symbol'] == 'AAPL') & 
    (aggregated_df['opening_strategy'].str.contains('spread|iron', case=False, na=False) | 
     aggregated_df['closing_strategy'].str.contains('spread|iron', case=False, na=False))
]

# Sort by expiration date and strike price
aapl_spreads = aapl_spreads.sort_values(['expiration_date', 'strike_price'])

# Initialize lists to store paired and unpaired orders
paired_orders = []
unpaired_opens = []

# Iterate through the orders
for _, order in aapl_spreads.iterrows():
    if pd.notna(order['opening_strategy']):
        # This is an opening order
        remaining_quantity = order['processed_quantity']
        matching_closes = []

        potential_closes = aapl_spreads[
            (aapl_spreads['closing_strategy'].notna()) &
            (aapl_spreads['expiration_date'] == order['expiration_date']) &
            (aapl_spreads['strike_price'] == order['strike_price']) &
            (aapl_spreads['option_type'] == order['option_type'])
        ]

        for _, close_order in potential_closes.iterrows():
            if remaining_quantity > 0:
                matched_quantity = min(remaining_quantity, close_order['processed_quantity'])
                matching_closes.append((close_order, matched_quantity))
                remaining_quantity -= matched_quantity

        if matching_closes:
            paired_orders.append((order, matching_closes))
            for close_order, _ in matching_closes:
                aapl_spreads = aapl_spreads[aapl_spreads.index != close_order.name]
        else:
            unpaired_opens.append(order)

    aapl_spreads = aapl_spreads[aapl_spreads.index != order.name]

# Print results
print(f"Paired orders: {len(paired_orders)}")
print(f"Unpaired opening orders: {len(unpaired_opens)}")

# Display first few paired orders
for i, (open_order, close_orders) in enumerate(paired_orders[:5]):
    print(f"\nPair {i+1}:")
    print(f"Open: {open_order['order_created_at']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['processed_quantity']}")
    total_closed = sum(close_order[1] for close_order in close_orders)
    print(f"Closed: {total_closed} out of {open_order['processed_quantity']}")
    for j, (close_order, matched_quantity) in enumerate(close_orders):
        print(f"  Close {j+1}: {close_order['order_created_at']} - {close_order['strike_price']} - {close_order['closing_strategy']} - Matched Quantity: {matched_quantity}")

# Display unpaired opening orders
print("\nUnpaired opening orders:")
for i, order in enumerate(unpaired_opens[:5]):
    print(f"{i+1}: {order['order_created_at']} - Exp: {order['expiration_date']} - {order['strike_price']} - {order['opening_strategy']} - Quantity: {order['processed_quantity']}")

# Calculate total opened and closed quantities
total_opened = sum(order['processed_quantity'] for order, _ in paired_orders) + sum(order['processed_quantity'] for order in unpaired_opens)
total_closed = sum(sum(close_order[1] for close_order in close_orders) for _, close_orders in paired_orders)

print(f"\nTotal opened quantity: {total_opened}")
print(f"Total closed quantity: {total_closed}")
print(f"Remaining open quantity: {total_opened - total_closed}")

Paired orders: 5
Unpaired opening orders: 1

Pair 1:
Open: 2023-12-19 15:59:35.91 - 202.5/202.5 - call_calendar_spread - Quantity: 1.0
Closed: 1.0 out of 1.0
  Close 1: 2023-12-22 18:58:08.92 - 202.5/202.5 - call_calendar_spread - Matched Quantity: 1.0

Pair 2:
Open: 2024-03-05 14:38:28.74 - 170.0/167.5 - short_put_spread - Quantity: 2.0
Closed: 2.0 out of 2.0
  Close 1: 2024-03-11 14:37:40.33 - 170.0/167.5 - short_put_spread - Matched Quantity: 2.0

Pair 3:
Open: 2024-03-21 15:13:48.01 - 172.5/170.0 - short_put_spread - Quantity: 4.0
Closed: 4.0 out of 4.0
  Close 1: 2024-03-21 17:33:25.67 - 172.5/170.0 - short_put_spread - Matched Quantity: 4.0

Pair 4:
Open: 2024-04-16 14:29:41.45 - 172.5/170.0 - short_call_spread - Quantity: 3.0
Closed: 3.0 out of 3.0
  Close 1: 2024-04-18 13:31:39.85 - 172.5/170.0 - short_call_spread - Matched Quantity: 3.0

Pair 5:
Open: 2024-05-16 14:59:20.30 - 192.5/190.0 - long_call_spread - Quantity: 1.0
Closed: 1.0 out of 1.0
  Close 1: 2024-05-17 18:37:35.5

# AAPL Spread Order Analysis

This cell analyzes AAPL spread orders by:

1. Filtering for AAPL spread/iron condor orders
2. Pairing opening orders with corresponding closing orders
3. Handling partial closings and multiple closing orders per opening order
4. Calculating total opened and closed quantities
5. Identifying unpaired (still open) orders

The analysis provides:
- Count of paired and unpaired orders
- Detailed view of first few paired orders (open and close details)
- List of unpaired (open) orders
- Summary of total opened, closed, and remaining open quantities

For unpaired opening orders, it searches for potential closing orders of individual legs by matching:
- Expiration date
- Any matching strike price
- AAPL as the underlying

This helps in understanding the current state of AAPL spread positions, including fully closed, partially closed, and still open trades, as well as identifying potential leg-by-leg closures.

In [93]:
# Assuming we're using the aggregated_df and aapl_spreads from the previous cell

# Convert strike_price to float when creating aggregated_df
aggregated_df = df.groupby('order_created_at').agg({
    'chain_symbol': 'first',
    'expiration_date': 'first',
    'strike_price': lambda x: [float(s) for s in sorted(x, key=abs, reverse=True)],
    'option_type': 'first',
    'direction': 'first',
    'order_quantity': 'first',
    'order_type': 'first',
    'opening_strategy': 'first',
    'closing_strategy': 'first',
    'price': 'first',
    'processed_quantity': 'first'
}).reset_index()

# Function to extract strike prices from the strike_price string
def extract_strikes(strike_string):
    return [float(s.strip('+-')) for s in strike_string.split('/')]

# Process unpaired opening orders
for i, open_order in enumerate(unpaired_opens):
    print(f"\nAnalyzing unpaired order {i+1}:")
    print(f"Open: {open_order['order_created_at']} - Exp: {open_order['expiration_date']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['processed_quantity']}")
    
    strikes = extract_strikes(open_order['strike_price'])
    
    potential_closes = aggregated_df[
        (aggregated_df['closing_strategy'].notna()) &
        (aggregated_df['expiration_date'] == open_order['expiration_date']) &
        (aggregated_df['chain_symbol'] == 'AAPL') &
        (aggregated_df['strike_price'].apply(lambda x: any(s in strikes for s in x)))
    ]
    
    if not potential_closes.empty:
        print("Potential closing orders for individual legs:")
        for _, close in potential_closes.iterrows():
            print(f"  Close: {close['order_created_at']} - Exp: {close['expiration_date']} - {close['strike_price']} - {close['closing_strategy']} - Quantity: {close['processed_quantity']}")
    else:
        print("No potential closing orders found for individual legs.")

# Calculate and print summary
total_unpaired_quantity = sum(order['processed_quantity'] for order in unpaired_opens)
print(f"\nTotal quantity of unpaired opening orders: {total_unpaired_quantity}")
print(f"Number of unpaired opening orders: {len(unpaired_opens)}")


Analyzing unpaired order 1:
Open: 2019-04-23 15:32:43.12 - Exp: 2019-05-03 - 205.0/202.5 - short_put_spread - Quantity: 2.0
Potential closing orders for individual legs:
  Close: 2019-04-24 19:12:28.83 - Exp: 2019-05-03 - [205.0] - short_put - Quantity: 2.0
  Close: 2019-04-24 19:12:53.83 - Exp: 2019-05-03 - [202.5] - long_put - Quantity: 2.0

Total quantity of unpaired opening orders: 2.0
Number of unpaired opening orders: 1


In [85]:
# Get all unique chain symbols
unique_symbols = aggregated_df['chain_symbol'].unique()

# Function to extract strike prices from the strike_price string
def extract_strikes(strike_string):
    return [float(s.strip('+-')) for s in strike_string.split('/')]

# Analyze each symbol
for symbol in unique_symbols:
    print(f"\n--- Analysis for {symbol} ---")
    
    # Filter for the current symbol's spread orders
    symbol_spreads = aggregated_df[
        (aggregated_df['chain_symbol'] == symbol) & 
        (aggregated_df['opening_strategy'].str.contains('spread|iron', case=False, na=False))
    ]
    
    # Find unpaired opening orders
    unpaired_opens = []
    for _, order in symbol_spreads.iterrows():
        closing_order = aggregated_df[
            (aggregated_df['chain_symbol'] == symbol) &
            (aggregated_df['closing_strategy'].str.contains('spread|iron', case=False, na=False)) &
            (aggregated_df['expiration_date'] == order['expiration_date']) &
            (aggregated_df['strike_price'] == order['strike_price'])
        ]
        if closing_order.empty:
            unpaired_opens.append(order)

    # Process unpaired opening orders
    for i, open_order in enumerate(unpaired_opens):
        print(f"\nAnalyzing unpaired order {i+1}:")
        print(f"Open: {open_order['order_created_at']} - Exp: {open_order['expiration_date']} - {open_order['strike_price']} - {open_order['opening_strategy']} - Quantity: {open_order['processed_quantity']}")
        
        strikes = set(open_order['strike_price'])
        
        potential_closes = aggregated_df[
            (aggregated_df['closing_strategy'].notna()) &
            (aggregated_df['expiration_date'] == open_order['expiration_date']) &
            (aggregated_df['chain_symbol'] == symbol) &
            (aggregated_df['strike_price'].apply(lambda x: any(s in strikes for s in x)))
        ]
        
        if not potential_closes.empty:
            print("Potential closing orders for individual legs:")
            for _, close in potential_closes.iterrows():
                print(f"  Close: {close['order_created_at']} - Exp: {close['expiration_date']} - {close['strike_price']} - {close['closing_strategy']} - Quantity: {close['processed_quantity']}")
        else:
            print("No potential closing orders found for individual legs.")

    # Calculate and print summary for this symbol
    total_unpaired_quantity = sum(order['processed_quantity'] for order in unpaired_opens)
    print(f"\nTotal quantity of unpaired opening orders for {symbol}: {total_unpaired_quantity}")
    print(f"Number of unpaired opening orders for {symbol}: {len(unpaired_opens)}")


--- Analysis for AAPL ---

Analyzing unpaired order 1:
Open: 2019-04-23 15:32:43.12 - Exp: 2019-05-03 - 205.0/202.5 - short_put_spread - Quantity: 2.0
Potential closing orders for individual legs:
  Close: 2019-04-24 19:12:53.83 - Exp: 2019-05-03 - 202.5 - long_put - Quantity: 2.0
  Close: 2019-04-24 19:12:28.83 - Exp: 2019-05-03 - 205.0 - short_put - Quantity: 2.0

Total quantity of unpaired opening orders for AAPL: 2.0
Number of unpaired opening orders for AAPL: 1

--- Analysis for ACB ---

Total quantity of unpaired opening orders for ACB: 0
Number of unpaired opening orders for ACB: 0

--- Analysis for ADI ---

Analyzing unpaired order 1:
Open: 2023-12-21 17:20:09.33 - Exp: 2023-12-29 - 197.5/197.5 - call_calendar_spread - Quantity: 1.0
No potential closing orders found for individual legs.

Total quantity of unpaired opening orders for ADI: 1.0
Number of unpaired opening orders for ADI: 1

--- Analysis for AI ---

Total quantity of unpaired opening orders for AI: 0
Number of unpa

In [74]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

# Fetch events for TSLA
events = r.get_events('NVDA')

# Convert the events to a DataFrame
events_df = pd.DataFrame(events)

# Select only the desired columns
events_df = events_df[['created_at', 'direction', 'quantity', 'total_cash_amount', 'type']]

# Initialize total cash amount
total_cash_amount = 0

# Sum total_cash_amount based on direction
for _, event in events_df.iterrows():
    if event['direction'] == 'debit':
        total_cash_amount -= float(event['total_cash_amount'])  # Subtract for debit
    elif event['direction'] == 'credit':
        total_cash_amount += float(event['total_cash_amount'])  # Add for credit

# Print the total cash amount
print(f"Total cash amount: {total_cash_amount}")

print(events_df)


Total cash amount: -5014.619999999995
                    created_at direction quantity total_cash_amount  \
0  2024-04-05T20:19:14.102872Z     debit  10.0000         890000.00   
1  2024-04-05T20:19:13.997059Z     debit  10.0000         900000.00   
2  2024-04-05T20:19:13.836079Z    credit  10.0000         897492.65   
3  2024-04-05T20:19:13.674223Z    credit  10.0000         887492.73   

         type  
0  assignment  
1  assignment  
2    exercise  
3    exercise  


In [81]:
# ... existing code ...
import pandas as pd
import json  # Use json to parse the string representation of lists/dictionaries

unique_symbols = aggregated_df['chain_symbol'].unique()
total_option_event_cost = 0  # Initialize total cash amount

# Create a list to store event data
event_data = []

for symbol in unique_symbols:
    events = r.get_events(symbol)  # Fetch events for the symbol
    for event in events:
        # Ignore rows with total_cash_amount = 0
        if float(event['total_cash_amount']) == 0 :
            continue
        
        # Append event details to the list
        event_data.append({
            'created_at': event['created_at'],
            'chain_symbol': symbol,
            'direction': event['direction'],
            'quantity': event['quantity'],
            'total_cash_amount': event['total_cash_amount'],
            'state': event['state'],
            'underlying_price': event['underlying_price']
        })
        if event['direction'] == 'credit':
            total_option_event_cost += float(event['total_cash_amount'])  # Add for exercise
        elif event['direction'] == 'debit':
            total_option_event_cost -= float(event['total_cash_amount'])  # Subtract for assignment

# Create a DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Save the events DataFrame to a new CSV file
events_df.to_csv('../output/option_events.csv', index=True)


# Print the total cash amount
print(f"Total cash amount: {total_option_event_cost}")


Total cash amount: -9115.660000000164


In [107]:
total =  total_option_cost + total_stock_cost + total_crypto_cost + abs(total_option_event_cost)
print(total)

8919.601299630365
